In [5]:
import json
import re 
import copy
from importlib import reload
import pandas as pd


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


## Configuration

This is controlled from `predicament.utils.config`. This tells us: what data is there, and some of the characteristics of that data.

We also have `predicament.utils.file_utils` which controls file, directory and time stamp information.



In [3]:
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import BASE_DATA_FOLDER
from predicament.utils.config import STUDY_DATA_FOLDER
from predicament.utils.config import MOTOR_MOVEMENT_DATA_FOLDER

from predicament.utils.config import DREEM_EDF_CHANNELS


from predicament.utils.config import E4_file_paths
from predicament.utils.config import VG_file_paths
from predicament.utils.config import VG_Hz
from predicament.utils.config import EEG_buffer
from predicament.utils.config import E4_buffer

print(f"BASE_DATA_FOLDER = {BASE_DATA_FOLDER}")
print(f"MOTOR_MOVEMENT_DATA_FOLDER = {MOTOR_MOVEMENT_DATA_FOLDER}")
print(f"E4_file_paths = {E4_file_paths}")


print(f"VG_file_paths = {VG_file_paths}")
print("\tThis is a dictionary of participant ID to edf file subpaths")

print()
print(f"DREEM_EDF_CHANNELS = {DREEM_EDF_CHANNELS}")
print(f"VG_Hz = {VG_Hz}")
print(f"EEG_buffer = {EEG_buffer}")
print(f"E4_buffer = {E4_buffer}")


base_data_folder = ../data
motor_movement_data_folder = ../data/EEG-Motor-Movement-Imagery-Dataset/
E4_file_paths = {'VG_01': './VG01/E4_8921_15_44/', 'VG_03': './VG03/E4_9921_12_16/', 'VG_05': './VG05/E4_9921_13_24/', 'VG_06': './VG06/E4_51021_13_33/', 'VG_07': './VG07/E4_51021_15_39/', 'VG_08': './VG08/E4_71021_10_42/', 'VG_09': './VG09/E4_11221_14_46/', 'VG_10': './VG10/E4_31221_11_17/', 'VH_01': './VH01/E4_61021_11_03/', 'VH_02': './VH02/E4_61021_13_59/', 'VH_03': './VH03/E4_11221_11_22/'}
VG_file_paths = {'VG_01': './VG01/8921_15_52.edf', 'VG_02': './VG02/7921_15_18.edf', 'VG_03': './VG03/90921_12_20.edf', 'VG_05': './VG05/90921_13_27.edf', 'VG_06': './VG06/51021_13_40.edf', 'VG_08': './VG08/71021_10_44.edf', 'VG_09': './VG09/11221_14_59.edf', 'VG_10': './VG10/31221_11_24.edf', 'VH_01': './VH01/61021_11_17.edf', 'VH_02': './VH02/61021_14_3.edf', 'VH_03': './VH03/11221_11_21.edf'}
	This is a dictionary of participant ID to edf file subpaths

DREEM_EDF_CHANNELS = ['Accelero Norm', '

## Creating a workable set of data objects

We need some timeseries data of EEG traces and some information regarding the human-level events that took place. Then we need to integrate the two.

In [ ]:
from Ray import EEG_data

EEG_files = EEG_data.read_all_VG_files()
EEG_files

In [ ]:
from Ray.data_load_save import load_info_from_csv
ev_details = load_info_from_csv()

In [ ]:
for partID, EEG in EEG_files.items():
    EEG.set_event_details(ev_details[partID])

In [ ]:
EEG_files['VG_01'].event_details
EEG_files['VG_01'].EEG_data


In [ ]:
from Ray.EEG_data import read_VG_file

# Understanding EEG_data_obj

# We'll call read_VG_file to create one such object.
# This requires a participant ID and a list of excluded channels
part_ID = 'VG_01'
exclude_channels = []
EEG_data_obj = read_VG_file(part_ID)

print(f"EEG_data_obj = {EEG_data_obj}")
print(f"list(EEG_data_obj.EEG_data.keys()) = {list(EEG_data_obj.EEG_data.keys())}")
channel_data = EEG_data_obj.EEG_data['EEG Fpz-O1']
print(f"type(channel_data) = {type(channel_data)}")
print(f"channel_data.shape = {channel_data.shape}")
print(f"channel_data[:10] = {channel_data[:10]}")


In [ ]:
from predicament.utils.config import DREEM_EDF_CHANNELS
part_ID = 'VG_02'
include_channels = ['Accelero Norm', 'EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8', 'Positiongram', 'PulseOxy Infrare', 'PulseOxy Red Hea', 'Respiration x', 'Respiration y', 'Respiration z']
EEG_data_obj2 = read_VG_file(part_ID, include_channels = DREEM_EDF_CHANNELS)


In [ ]:
for ch_name, data in EEG_data_obj2.EEG_data.items():
    print(f"Channel {ch_name} has {len(data)} items")

In [ ]:
import pandas as pd
event_details_path = '../data/CARE_HOME_DATA/event_details.csv'
exp_info_path = '../data/CARE_HOME_DATA/exp_info.csv'

event_details_df = pd.read_csv(event_details_path, index_col=0)
exp_info_df = pd.read_csv(exp_info_path, index_col=0)

part_list = list(exp_info_df.index)
ev_details = {}

for partID in part_list:
    ev_detail = Event_time_details(partID)
    ev_detail.set_exp_datetime(exp_info_df.loc[partID]['date'], exp_info_df.loc[partID]['exp_start'])    
    for event in event_details_df.loc[partID,].iterrows():
        start = str(event[1]['start']) if event[1]['start'] != 'None' else None
        end = str(event[1]['end']) if event[1]['end'] != 'None' else None
        print(f"partID: end = {partID}:{end}")
        ev_detail.set_event(event[1]['event'], start, end, event[1]['valid'])
    ev_details[partID] = ev_detail


In [ ]:
partID = 'VG_01'
for event in event_details_df.loc[partID].iterrows():
    print(event)
float.nan

In [ ]:
import mne
VG_file_path = os.path.join(STUDY_DATA_FOLDER, VG_file_paths[part_ID])
mne_Raw_obj = mne.io.read_raw_edf(VG_file_path, exclude =[])
print(type(mne_Raw_obj))
print(f"mne_Raw_obj.ch_names = {mne_Raw_obj.ch_names}")
mne_Raw_obj.info

In [ ]:
to_pick = ['EEG Fpz-O1', 'EEG Fpz-O2', 'EEG Fpz-F7', 'EEG F8-F7', 'EEG F7-01', 'EEG F8-O2', 'EEG Fpz-F8']
mne_Raw_obj.pick(to_pick)
print(f"mne_Raw_obj.ch_names = {mne_Raw_obj.ch_names}")

mne_Raw_obj

In [ ]:
from Ray import EEG_data, E4_data
from predicament.utils.config import data_folder, VG_file_paths, VG_Hz, EEG_buffer
from Ray.Event_details import Event_time_details
from Ray.data_load_save import save_info_to_csv

save_info_to_csv()

In [ ]:
# resulting output from following call is
# dictionary str to type EEG_data_obj
# where EEG_data_obj is defined in Ray.EEG_data

all_eeg_data = EEG_data.read_all_VG_files()
event_details_dic = {
    'partID': [],
    'event': [],
    'start': [],
    'end': [],
    'valid': []
}

exp_info_dic = {
    'partID': [],
    'date': [],
    'exp_start': []
}

In [ ]:
all_eeg_data['VG_05'].event_details.events_info.items()

In [ ]:
event_ids = dict(familiar_music=1, wildlife_video=2, family_inter=3, Tchaikovsky=4, exper_video=5)
event_ids['familiar_music']

In [ ]:
from Ray.data_load_save import set_up
EEG_files, _ = set_up(isE4 = False)

In [ ]:
EEG_files['VG_01'].event_details.events_info

## Train/test data (evaluation data)


The method `prepare_evaluation_data.py` loads the obervational and eeg data and integrates it to produce some train and test data in 'data/EEG-Motor-Movement-Imagery-Dataset/5classes_part/{train,test}_{set,labels}.csv' (at time of writing). Let's investigate this.


In [ ]:
import pandas as pd
test_set_fname = '../data/EEG-Motor-Movement-Imagery-Dataset/5classes_part/test_set.csv'

test_set_df = pd.read_csv(test_set_fname, header=None)

test_set_df

In [ ]:
4096/250

## Time-series data



In [ ]:
from Ray import data_load_save
EEG_files, E4_files = data_load_save.set_up(
    isEEG=True, isE4=False)
    # file_utils.gen_train_test_valid(EEG_files)

In [ ]:
import matplotlib.pyplot as plt
# Visualize VG data (EEG 250Hz)
EEG_partID, EEG_channel, EEG_event = 'VG_06', 'EEG Fpz-O1', 'familiar_music'
plt.plot(EEG_files[EEG_partID].get_EEG_by_channel_and_event(EEG_channel, EEG_event), label = EEG_channel)
plt.plot(EEG_files[EEG_partID].get_EEG_by_channel_and_event('EEG F7-01', EEG_event), label = 'F7-01')
plt.plot(EEG_files[EEG_partID].get_EEG_by_channel_and_event('EEG Fpz-F8', EEG_event), label = 'Fpz-F8')

## Study Events




In [6]:
from predicament.utils.config import EVENT_DETAILS_PATH
from predicament.utils.config import EXP_INFO_PATH
event_details_path = EVENT_DETAILS_PATH
exp_info_path = EXP_INFO_PATH
print(f"event_details_path = {event_details_path}")
event_details_df = pd.read_csv(event_details_path, index_col=0)
print(f"event_details_df.loc['VG_01'] = {event_details_df.loc['VG_01']}")
exp_info_df = pd.read_csv(exp_info_path, index_col=0)

event_details_path = ../data/CARE_HOME_DATA/./event_details.csv
event_details_df.loc['VG_01'] =                  event     start       end  valid
partID                                           
VG_01            setup  15:52:00      None   True
VG_01         baseline  15:52:00      None   True
VG_01      exper_video  16:02:00  16:10:00   True
VG_01   wildlife_video  15:56:00  16:01:00   True
VG_01   familiar_music  16:16:00  16:21:00   True
VG_01      Tchaikovsky  16:11:00  16:16:00   True
VG_01     family_inter  16:22:00  16:27:00   True


In [8]:
event_details_df.loc['VG_01',]

,event,start,end,valid
partID,,,,
VG_01,setup,15:52:00,None,True
VG_01,baseline,15:52:00,None,True
VG_01,exper_video,16:02:00,16:10:00,True
VG_01,wildlife_video,15:56:00,16:01:00,True
VG_01,familiar_music,16:16:00,16:21:00,True
VG_01,Tchaikovsky,16:11:00,16:16:00,True
VG_01,family_inter,16:22:00,16:27:00,True


In [13]:
for row in event_details_df.loc['VG_01',].iterrows():
    print(row[1]['event'])

setup
baseline
exper_video
wildlife_video
familiar_music
Tchaikovsky
family_inter
